In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [15]:
from Dataset.Dataset import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.kerasModels import KerasModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report


import tensorflow as tf
print(tf.version.VERSION)
from tensorflow.keras.losses import binary_crossentropy

2.2.0


In [16]:
#Load Dataset
dataset = CSVLoader('preprocessed_dataset_wfoodb.csv', 'Smiles', ['Class'], 'ID')#, chunk_size=4000)
print(dataset.get_shape())

(23290,) (23290,) (0,) (23290,)
((23290,), (23290,), (0,), (23290,))


In [17]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [14:55:40] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [14:55:43] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [14:55:43] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [14:55:57] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
(23286,) (23286,) (23286, 1024) (23286,)


((23286,), (23286,), (23286, 1024), (23286,))

In [18]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

(23286,) (23286,) (23286, 49) (23286,)


((23286,), (23286,), (23286, 49), (23286,))

In [19]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

In [20]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np


input_dim = train_dataset.features.shape[1]


def create_model(optimizer='adam', dropout=0.5, input_dim=input_dim):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [7]:
#print(type(keras_model))
#from sklearn.model_selection import GridSearchCV 
#model = KerasClassifier(build_fn=create_model, epochs=15, batch_size=10, verbose=1, optimizer='adam')

#model.fit(train_dataset.features, train_dataset.y)
#params={'optimizer':['adam', 'rsmprop'],  
#        'nb_epoch':[200, 100, 300, 400]}

#gs=GridSearchCV(estimator=model, param_grid=params, cv=3) 
# now fit the dataset to the GridSearchCV object.  
#gs = gs.fit(train_dataset.features, train_dataset.y)

#print(gs)
#print(type(model))

#seed = 7
#np.random.seed(seed)
#print(1)
#kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
#print(2)
#results = cross_val_score(model, train_dataset.features, train_dataset.y, cv=kfold)
#print(3)
#print(results)

#model = KerasModel(keras_model, binary_crossentropy)

In [8]:
from models.kerasModels import KerasModel

input_dim = train_dataset.features.shape[1]
print(input_dim)
model = KerasModel(create_model, epochs = 150, verbose=1, optimizer='adam')#, input_dim=input_dim)



49


In [9]:
print(train_dataset.features.shape, train_dataset.y.shape)


model.fit(train_dataset)#.features, train_dataset.y)#, nb_epoch=50)


(13968, 49) (13968,)
kerasModels fit!
Epoch 1/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.2261 - accuracy: 0.9382
Epoch 2/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1862 - accuracy: 0.9434
Epoch 3/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1792 - accuracy: 0.9434
Epoch 4/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1715 - accuracy: 0.9442
Epoch 5/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1719 - accuracy: 0.9447
Epoch 6/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1698 - accuracy: 0.9444
Epoch 7/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1685 - accuracy: 0.9452
Epoch 8/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1681 - accuracy: 0.9443
Epoch 9/10
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1653 - accuracy: 0.9458
Epoch 10/10
1397/1397 [=====================

In [10]:
metrics = [Metric(roc_auc_score), 
           Metric(precision_score), 
           Metric(accuracy_score), 
           Metric(confusion_matrix), 
           Metric(classification_report)]

print('training set score:', model.evaluate(train_dataset, metrics))
print('test set score:', model.evaluate(test_dataset, metrics))

Models Class line 158 --> evaluator
Instructions for updating:
Please use `model.predict()` instead.
1397/1397 [==============================] - 1s 753us/step
roc_auc_score: 
 0.5094816687737042
precision_score: 
 1.0
accuracy_score: 
 0.9444444444444444
confusion_matrix: 
 [[13177     0]
 [  776    15]]
classification_report: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97     13177
           1       1.00      0.02      0.04       791

    accuracy                           0.94     13968
   macro avg       0.97      0.51      0.50     13968
weighted avg       0.95      0.94      0.92     13968

training set score: {'roc_auc_score': 0.5094816687737042, 'precision_score': 1.0, 'accuracy_score': 0.9444444444444444, 'confusion_matrix': 3492.0, 'classification_report': None}
Models Class line 158 --> evaluator
466/466 [==============================] - 0s 746us/step
roc_auc_score: 
 0.5113365901454794
precision_score: 
 0.857142857142

In [14]:
from parameterOptimization.HyperparameterOpt import GridHyperparamOpt
#Hyperparameter Optimization
optimizer = GridHyperparamOpt(create_model)

params_dict = {'optimizer' : ['adam', 'rmsprop'],
              'dropout' : [0.2, 0.4, 0.5]}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_dataset, 
                                                                        valid_dataset, Metric(roc_auc_score),
                                                                       cv=5, n_jobs=1, verbose=3)

print('#################')
print(best_hyperparams)
print(best_model)

Fitting 6 random models from a space of 6 possible models.
Fitting model 1/6
hyperparameters: {'optimizer': 'adam', 'dropout': 0.2}
Using accuracy instead and  roc_auc_score  on validation!
 

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] dropout=0.2, optimizer=adam .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


350/350 [==============================] - 0s 1ms/step - loss: 0.3040 - accuracy: 0.9102
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[CV] ......... dropout=0.2, optimizer=adam, score=1.000, total=   0.9s
[CV] dropout=0.2, optimizer=adam .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


350/350 [==============================] - 0s 1ms/step - loss: 0.2470 - accuracy: 0.9292
[CV] ......... dropout=0.2, optimizer=adam, score=1.000, total=   0.9s
[CV] dropout=0.2, optimizer=adam .....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.9s remaining:    0.0s


350/350 [==============================] - 1s 1ms/step - loss: 0.2931 - accuracy: 0.9186
[CV] ......... dropout=0.2, optimizer=adam, score=1.000, total=   1.0s
[CV] dropout=0.2, optimizer=adam .....................................
350/350 [==============================] - 0s 1ms/step - loss: 0.3250 - accuracy: 0.8791
[CV] ......... dropout=0.2, optimizer=adam, score=1.000, total=   0.9s
[CV] dropout=0.2, optimizer=adam .....................................
350/350 [==============================] - 0s 1ms/step - loss: 0.1496 - accuracy: 0.9540
[CV] ......... dropout=0.2, optimizer=adam, score=0.717, total=   0.9s
[CV] dropout=0.2, optimizer=rmsprop ..................................
350/350 [==============================] - 1s 2ms/step - loss: 0.2615 - accuracy: 0.9276
[CV] ...... dropout=0.2, optimizer=rmsprop, score=1.000, total=   1.2s
[CV] dropout=0.2, optimizer=rmsprop ..................................
350/350 [==============================] - 1s 2ms/step - loss: 0.2508 - accu

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   30.8s finished


437/437 [==============================] - 1s 1ms/step - loss: 0.2239 - accuracy: 0.9367

Evaluating best model on the validation dataset...
 

kerasModels fit!
Models Class line 158 --> evaluator
roc_auc_score: 
 0.5689426403394962
valid_dataset:  {'roc_auc_score': 0.5689426403394962}
Best: 0.943358 using {'dropout': 0.2, 'optimizer': 'adam'}
0.943358 (0.113283) with: {'dropout': 0.2, 'optimizer': 'adam'}
0.943358 (0.113283) with: {'dropout': 0.2, 'optimizer': 'rmsprop'}
0.943358 (0.113283) with: {'dropout': 0.4, 'optimizer': 'adam'}
0.943358 (0.113283) with: {'dropout': 0.4, 'optimizer': 'rmsprop'}
0.943358 (0.113283) with: {'dropout': 0.5, 'optimizer': 'adam'}
0.943358 (0.113283) with: {'dropout': 0.5, 'optimizer': 'rmsprop'}
#################
{'dropout': 0.2, 'optimizer': 'adam'}


In [ ]:
#pip install tensorflow==2.2.0

In [ ]:

#import tensorflow as tf
#tf.version.VERSION

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()